## Speed test

In this notebook we compare the execution speed of the different pricing model and the imporvement made to speed them up.

In [39]:
import numpy as np
import pandas as pd
import ast
import os
import sys
from os.path import dirname as up

# Important directories
code_dir = os.path.dirname(os.getcwd())
deep_cal_dir = os.path.dirname(os.path.dirname(os.getcwd()))

# Allows to import my own module
sys.path.insert(0, code_dir)

from pricing.liftedheston import Pricer_Lifted_Heston
from pricing.legacyliftedheston import Legacy_Pricer_Lifted_Heston
from pricing.linear_iv import linear
from pricing.newton_iv import newton_raphson
from pricing.brent_iv import brent, call_BS
from pricing.heston_fourier import price_heston_fourier
from pricing.legacy import price_heston_mc_legacy
from pricing.heston_mc import price_heston_mc

### Linear implied volatility

In [2]:
%timeit linear(S_=0.8,K_=1.,r_=0.03,tau_=0.8,price_=3)

11.8 µs ± 407 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Newton implied volatility 

In [3]:
%timeit newton_raphson(S_=0.8,K_=1.,r_=0.03,tau_=0.8, sigma0_= 0.1,price_=3, epsilon_=1e-5)

647 µs ± 19.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Brent implied Volatility 

In [4]:
%timeit brent(S_=0.8,K_=1.,r_=0.03,tau_=0.8, sigma0_= 0.1,sigma1_=0.5,price_=3, epsilon_=1e-5)

1.06 ms ± 43.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### BS

In [75]:
import scipy.stats as stats
import scipy as scp
def call_option(S_, K_, r_,sigma_ ,tau_ ):
    d1_ = 1 / (sigma_ * np.sqrt(tau_)) * (np.log(S_/K_) + (r_ + sigma_**2/2) * tau_)
    d2_ = d1_ - sigma_ * np.sqrt(tau_)
    return stats.norm.cdf(d1_) * S_ - stats.norm.cdf(d2_) * K_ * np.exp(-r_ *tau_) -2.352700679476849e-06

In [40]:
%timeit call_BS(S_=1, K_=0.9, r_=-0.7,sigma_=0.221 ,tau_=0.5,price=0)

158 µs ± 5.31 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [79]:
scp.optimize.brentq(lambda x: call_option(S_=1, K_=1.5, r_=-0.7,sigma_=x ,tau_=.5),-100,100)

0.26973093490803707

### Heston Fourier

In [5]:
%timeit price_heston_fourier(K_=1.,alpha_=1.,r_=0.05,tau_=0.8,kappa_=1.5,S0_=0.8,theta_=0.1,rho_=-0.05,sigma_=0.3,V0_=0.1,L_=50)

4.2 ms ± 163 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [74]:
price_heston_fourier(K_=1.5,alpha_=1.,r_=0.03,tau_=0.5,kappa_=2,S0_=1,theta_=0.04,rho_=-0.7,sigma_=0.5,V0_=0.04,L_=50)
# price_heston_fourier(90,1,0.03,0.5,2,100,0.04,-0.7,0.5,0.04,50)

2.352700679476849e-06

### Heston Monte Carlo

Legacy version

In [6]:
%timeit price_heston_mc_legacy(kappa_=1.5,theta_=0.1,sigma_=0.3,rho_=-0.05,r_=0.05,T_=0.8,L_=100,V0_=0.1,S0_=0.8,K0_=1,N_=1000)

831 ms ± 27.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
price_heston_mc_legacy(kappa_=2,theta_=0.04,sigma_=0.04,rho_=-0.7,r_=0.03,T_=0.5,L_=100,V0_=0.04,S0_=1,K0_=0.9,N_=1000)

0.12812334028991254

Speed up mc

In [25]:
%timeit price_heston_mc(kappa_=2,theta_=0.04,sigma_=0.04,rho_=-0.7,r_=0.03,T_=0.5,L_=100,V0_=0.04,S0_=1,K0_=0.9,N_=1000)

47.2 ms ± 1.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
price_heston_mc(kappa_=2,theta_=0.04,sigma_=0.04,rho_=-0.7,r_=0.03,T_=0.5,L_=100,V0_=0.04,S0_=1,K0_=0.9,N_=1000)

0.13216876097762117

In [28]:
%timeit Pricer_Lifted_Heston(0.9,0.03,1,0.5,rho=-0.7,lamb=0.04,theta=0.04,nu=0.3,V0=0.04,N=20,rN=2.5,alpha=0.1+1/2,M=200,L_=50)

261 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
Pricer_Lifted_Heston(0.9,0.03,1,0.5,rho=-0.7,lamb=0.04,theta=0.04,nu=0.1,V0=0.04,N=20,rN=2.5,alpha=0.1+1/2,M=200,L_=50)

array([0.11823299])

In [9]:
%timeit Legacy_Pricer_Lifted_Heston(1,0.03,0.8,0.8,rho=-0.7,lamb=0.3,theta=0.02,nu=0.3,V0=0.02,N=20,rN=2.5,alpha=0.1+1/2,M=200,L_=50)

828 ms ± 4.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
